In [223]:
import tensorflow as tf
from tensorflow import keras
from keras import utils
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Input, Dense, Flatten, Concatenate
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

import os
import random
from copy import deepcopy
import pickle

In [41]:
np.random.seed(3)

In [42]:
fashion_mnist = keras.datasets.fashion_mnist
((x_train, y_train), (x_test, y_test)) = fashion_mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000, 10)
(10000, 28, 28)
(10000, 10)


In [43]:
class Random_Finetune_ResNet50():
    def __init__(self, input_shape):

        self.fitness = 0
        # self.loss = 1000
        
        IMG_SHAPE = input_shape + (3,)
        self.base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
        sample_arr = [True, False]
        self.bool_arr = np.random.choice(sample_arr, size=len(self.base_model.layers))
        self.update_trainable()
        # self.base_model.trainable = True
        # for idx, i in enumerate(self.base_model.layers):
        #     i.trainable = self.bool_arr[idx]
    
    def update_trainable(self, bool_arr=None):
        if bool_arr is not None: 
            self.bool_arr = bool_arr
        self.base_model.trainable = True
        for idx, i in enumerate(self.base_model.layers):
            i.trainable = self.bool_arr[idx]
        
    def forward(self, learning_rate=0.001):
        inputs = Input((28, 28, 1))
        resized_x = tf.keras.layers.experimental.preprocessing.Resizing(32, 32)(inputs)
        first_conv_layer = Conv2D(3, 1, padding='same', activation=None)(resized_x)

        x = self.base_model(first_conv_layer, training = False)
        x = Flatten()(x)
        outputs = Dense(10, activation = 'softmax')(x)

        model = tf.keras.Model(inputs, outputs, name="fashion_mnist_resnet50_model")

        # 'categorical_crossentropy'은 y[0]=[0, 0, 0, 0, 0, 0, 0, 0, 1], y[1, 0, 0, 0, 0, 0, 0, 0, 0]과 같이 one-hot-encoding label일 경우에 사용
        model.compile(loss="categorical_crossentropy", 
        optimizer=tf.keras.optimizers.Adam(learning_rate= learning_rate), 
        metrics=['accuracy'])
        
        return model
    
    def train_model(self, model, train_data, train_targets, validation_data=(x_test, y_test), epochs=20, batch_size=256):
    
        early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
        checkpoint_best_path = 'model_checkpoints_best/checkpoint'
        checkpoint_best = ModelCheckpoint(filepath=checkpoint_best_path,
                                        save_weights_only=True,
                                        save_freq='epoch',
                                        monitor='val_accuracy',
                                        save_best_only=True,
                                        verbose=1)
        history = model.fit(train_data, train_targets,
                        validation_data = validation_data,
                        epochs = epochs,
                        batch_size = batch_size,
                        verbose = 1,
                        callbacks=[early])
        return history

In [44]:
N_POPULATION = 10
N_BEST = 6
N_CHILDREN = 5
PROB_MUTATION = 0.04

epoch = 3
batch_size =256

In [45]:
genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(N_POPULATION)]
best_genomes = None

In [46]:
(genomes[0].bool_arr ==genomes[1].bool_arr).all()

False

In [30]:
epoch = 5

genome = genomes[0]
model = genome.forward(0.0001)
history = genome.train_model(model, x_train, y_train, (x_test, y_test), epoch, 256)
fitness = history.history['val_accuracy']

print(fitness)
sorted_fitness = sorted(fitness, reverse=True)
print(sorted_fitness)

genome.fitness = sorted_fitness[0]

print(genomes[0].fitness)
print(genomes[1].fitness)

Epoch 1/5
235/235 [==============================] - 23s 77ms/step - loss: 1.2382 - accuracy: 0.5092 - val_loss: 0.8256 - val_accuracy: 0.6602
Epoch 2/5
235/235 [==============================] - 16s 68ms/step - loss: 0.7298 - accuracy: 0.7161 - val_loss: 0.8094 - val_accuracy: 0.6948
Epoch 3/5
235/235 [==============================] - 16s 69ms/step - loss: 0.6146 - accuracy: 0.7604 - val_loss: 0.5666 - val_accuracy: 0.7838
Epoch 4/5
235/235 [==============================] - 16s 68ms/step - loss: 0.5563 - accuracy: 0.7853 - val_loss: 0.5439 - val_accuracy: 0.7909
Epoch 5/5
235/235 [==============================] - 16s 68ms/step - loss: 0.5075 - accuracy: 0.8051 - val_loss: 0.4931 - val_accuracy: 0.8101
[0.6601999998092651, 0.6948000192642212, 0.7838000059127808, 0.7908999919891357, 0.8101000189781189]
[0.8101000189781189, 0.7908999919891357, 0.7838000059127808, 0.6948000192642212, 0.6601999998092651]
0.8101000189781189
0


In [47]:
# fitness는 TOP val_accuracy로 구성
# score는 val_loss로 구성
# genomes에 최종 학습이 끝난 epoch마다의 val_accuracy, bool_arr(layer별 얼릴 것인가 학습할 것인가) 정보가 포함되고 정렬됨

n_gen = 0

n_gen += 1

for i, genome in enumerate(genomes):

    genome = genomes[i]
    model = genome.forward(0.0001)
    history = genome.train_model(model, x_train, y_train, (x_test, y_test), epoch, batch_size)
    fitness = history.history['val_accuracy']
    sorted_fitness = sorted(fitness, reverse=True)
    genome.fitness = sorted_fitness[0]

    print('Generation #%s, Genome #%s, Fitness: %s, Best Fitness: %s' % (n_gen, i, fitness, genome.fitness))

for i, genome in enumerate(genomes):
    print("===== Generaton #%s\tAll Fitness %s =====" % (n_gen, genomes[i].fitness))

if best_genomes is not None:
    genomes.extend(best_genomes)
genomes.sort(key=lambda x: x.fitness, reverse=True)

print('===== Generaton #%s\tBest Fitness %s =====' % (n_gen, genomes[0].fitness))

Epoch 1/3
235/235 [==============================] - 23s 75ms/step - loss: 1.8918 - accuracy: 0.2786 - val_loss: 1.4173 - val_accuracy: 0.4417
Epoch 2/3
235/235 [==============================] - 17s 71ms/step - loss: 0.8749 - accuracy: 0.6588 - val_loss: 0.6316 - val_accuracy: 0.7602
Epoch 3/3
235/235 [==============================] - 17s 71ms/step - loss: 0.6389 - accuracy: 0.7559 - val_loss: 0.5325 - val_accuracy: 0.7918
Generation #1, Genome #0, Fitness: [0.4417000114917755, 0.760200023651123, 0.7918000221252441], Best Fitness: 0.7918000221252441
Epoch 1/3
235/235 [==============================] - 23s 73ms/step - loss: 1.4972 - accuracy: 0.4287 - val_loss: 0.8490 - val_accuracy: 0.6632
Epoch 2/3
235/235 [==============================] - 16s 67ms/step - loss: 0.7276 - accuracy: 0.7147 - val_loss: 0.7533 - val_accuracy: 0.7016
Epoch 3/3
235/235 [==============================] - 16s 67ms/step - loss: 0.6388 - accuracy: 0.7505 - val_loss: 0.5933 - val_accuracy: 0.7718
Generation #1

In [58]:
for i, genome in enumerate(genomes):
    print("===== Generaton #%s\tGenome #%s : Fitness %s =====" % (n_gen, i, genome.fitness))

===== Generaton #1	Genome #0 : Fitness 0.7947999835014343 =====
===== Generaton #1	Genome #1 : Fitness 0.7918000221252441 =====
===== Generaton #1	Genome #2 : Fitness 0.7842000126838684 =====
===== Generaton #1	Genome #3 : Fitness 0.7771999835968018 =====
===== Generaton #1	Genome #4 : Fitness 0.7742000222206116 =====
===== Generaton #1	Genome #5 : Fitness 0.7717999815940857 =====
===== Generaton #1	Genome #6 : Fitness 0.7487000226974487 =====
===== Generaton #1	Genome #7 : Fitness 0.7425000071525574 =====
===== Generaton #1	Genome #8 : Fitness 0.7354000210762024 =====
===== Generaton #1	Genome #9 : Fitness 0.7182000279426575 =====


In [71]:
accuracy = np.array([])
bool_arr = []

for i, genome in enumerate(genomes):
    print("===== Generaton #%s\tGenome #%s : Fitness %s =====" % (n_gen, i, genome.fitness))
    # print(type(genome.bool_arr))
    # print(genome.bool_arr)
    accuracy = np.append(accuracy, genome.fitness)
    # bool_arr = np.append(bool_arr, genome.bool_arr)
    bool_arr.append(genome.bool_arr)

bool_arr = np.asarray(bool_arr)

print(accuracy.shape)
print(bool_arr.shape)

===== Generaton #1	Genome #0 : Fitness 0.7947999835014343 =====
===== Generaton #1	Genome #1 : Fitness 0.7918000221252441 =====
===== Generaton #1	Genome #2 : Fitness 0.7842000126838684 =====
===== Generaton #1	Genome #3 : Fitness 0.7771999835968018 =====
===== Generaton #1	Genome #4 : Fitness 0.7742000222206116 =====
===== Generaton #1	Genome #5 : Fitness 0.7717999815940857 =====
===== Generaton #1	Genome #6 : Fitness 0.7487000226974487 =====
===== Generaton #1	Genome #7 : Fitness 0.7425000071525574 =====
===== Generaton #1	Genome #8 : Fitness 0.7354000210762024 =====
===== Generaton #1	Genome #9 : Fitness 0.7182000279426575 =====
(10,)
(10, 175)


In [78]:
winner_acc = accuracy[:N_BEST]
winner_bool_arr = bool_arr[:N_BEST]

populationWinnerData = zip(winner_acc, winner_bool_arr)

print(populationWinnerData)

In [101]:
winner_acc = accuracy[:N_BEST]
winner_bool_arr = bool_arr[:N_BEST]

a_genome = deepcopy(winner_bool_arr[0])
b_genome = deepcopy(winner_bool_arr[1])

new_genome = []
cut = np.random.randint(0, len(winner_bool_arr[0]))
new_genome.extend(a_genome[:cut])
new_genome.extend(b_genome[cut:])

children = np.asarray(new_genome)

print(cut)
print(len(children), type(children))

# print(crossover0.bool_arr is new_genome)

# crossover0.set_bool_arr(new_genome)
# print(crossover0.bool_arr is new_genome)

172
175 <class 'numpy.ndarray'>


In [102]:
flag = np.random.randint(0, len(winner_acc), size=2)
print(flag)
print(flag[0], flag[-1])

[2 3]
2 3


In [226]:
def crossover_sequentail(iteration, winner_acc, winner_bool_arr, class_instance_arr):
    
    # max part는 추후 수정 예정
    max = len(winner_acc)
    if max < iteration:
        for i in range(iteration-max):
            winner_acc = np.append(winner_acc, winner_acc[i])
            winner_bool_arr = np.append(winner_bool_arr, winner_bool_arr[i])

    for i, genome in enumerate(class_instance_arr):

        a_genome = deepcopy(winner_bool_arr[i])
        b_genome = deepcopy(winner_bool_arr[i-1])

        new_genome = []
        cut = np.random.randint(0, len(winner_bool_arr[0]))
        new_genome.extend(a_genome[:cut])
        new_genome.extend(b_genome[cut:])

        children = np.asarray(new_genome)
        genome.update_trainable(bool_arr=children)
        avg_fitness = (winner_acc[i] + winner_acc[i-1])/2
        genome.fitness = avg_fitness

        print('Generation #%s, Crossover_sequence Genome #%s Created Bool_Array: %s layers, Predicted Fitness: %s, Done' % (n_gen, i, len(children), genome.fitness))


def crossover_random(iteration, winner_acc, winner_bool_arr, class_instance_arr):

    # max part는 추후 수정 예정
    max = len(winner_acc)
    if max < iteration:
        for i in range(iteration-max):
            winner_acc = np.append(winner_acc, winner_acc[i])
            winner_bool_arr = np.append(winner_bool_arr, winner_bool_arr[i])
    
    for i, genome in enumerate(class_instance_arr):

        flag = np.random.randint(0, len(winner_acc), size=2)

        a_genome = deepcopy(winner_bool_arr[flag[0]])
        b_genome = deepcopy(winner_bool_arr[flag[-1]])

        new_genome = []
        cut = np.random.randint(0, len(winner_bool_arr[0]))
        new_genome.extend(a_genome[:cut])
        new_genome.extend(b_genome[cut:])

        children = np.asarray(new_genome)
        genome.update_trainable(bool_arr=children)
        avg_fitness = (winner_acc[i] + winner_acc[i-1])/2
        genome.fitness = avg_fitness

        print('Generation #%s, Crossover_Random Genome #%s Created Bool_Array: %s layers, Predicted Fitness: %s, Done' % (n_gen, i, len(children), genome.fitness))


In [111]:
# crossover Class Level with i and i-1
crossovers = [Random_Finetune_ResNet50((32,32)) for _ in range(N_CHILDREN)]

crossover_bool_arr = []
for i, genome in enumerate(crossovers):
    crossover_bool_arr.append(genome.bool_arr)
    print(crossover_bool_arr[i] is crossovers[i].bool_arr)

crossover_sequentail(winner_acc, winner_bool_arr, crossovers)
print(crossover_bool_arr[0] is crossovers[0].bool_arr)

True
True
True
True
True
Generation #1, Crossover Genome #0 Created Bool_Array: 175 layers, Predicted Fitness: 0.784500002861023, Done
Generation #1, Crossover Genome #1 Created Bool_Array: 175 layers, Predicted Fitness: 0.7933000028133392, Done
Generation #1, Crossover Genome #2 Created Bool_Array: 175 layers, Predicted Fitness: 0.7880000174045563, Done
Generation #1, Crossover Genome #3 Created Bool_Array: 175 layers, Predicted Fitness: 0.7806999981403351, Done
Generation #1, Crossover Genome #4 Created Bool_Array: 175 layers, Predicted Fitness: 0.7757000029087067, Done
False


In [203]:
# crossover Class Level with RANDOM
crossovers = [Random_Finetune_ResNet50((32,32)) for _ in range(N_CHILDREN)]

crossover_bool_arr = []
for i, genome in enumerate(crossovers):
    crossover_bool_arr.append(genome.bool_arr)
    print(crossover_bool_arr[i] is crossovers[i].bool_arr)

crossover_random(winner_acc, winner_bool_arr, crossovers)
print(crossover_bool_arr[0] is crossovers[0].bool_arr)

True
True
True
True
True
Generation #1, Crossover Genome #0 Created Bool_Array: 175 layers, Predicted Fitness: 0.784500002861023, Done
Generation #1, Crossover Genome #1 Created Bool_Array: 175 layers, Predicted Fitness: 0.7933000028133392, Done
Generation #1, Crossover Genome #2 Created Bool_Array: 175 layers, Predicted Fitness: 0.7880000174045563, Done
Generation #1, Crossover Genome #3 Created Bool_Array: 175 layers, Predicted Fitness: 0.7806999981403351, Done
Generation #1, Crossover Genome #4 Created Bool_Array: 175 layers, Predicted Fitness: 0.7757000029087067, Done
False


In [216]:
new_genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(N_POPULATION)]

crossover_sequentail(winner_acc, winner_bool_arr, new_genomes[:N_CHILDREN])
crossover_sequentail(winner_acc, winner_bool_arr, new_genomes[N_CHILDREN:])

Generation #1, Crossover Genome #0 Created Bool_Array: 175 layers, Predicted Fitness: 0.784500002861023, Done
Generation #1, Crossover Genome #1 Created Bool_Array: 175 layers, Predicted Fitness: 0.7933000028133392, Done
Generation #1, Crossover Genome #2 Created Bool_Array: 175 layers, Predicted Fitness: 0.7880000174045563, Done
Generation #1, Crossover Genome #3 Created Bool_Array: 175 layers, Predicted Fitness: 0.7806999981403351, Done
Generation #1, Crossover Genome #4 Created Bool_Array: 175 layers, Predicted Fitness: 0.7757000029087067, Done
Generation #1, Crossover Genome #0 Created Bool_Array: 175 layers, Predicted Fitness: 0.784500002861023, Done
Generation #1, Crossover Genome #1 Created Bool_Array: 175 layers, Predicted Fitness: 0.7933000028133392, Done
Generation #1, Crossover Genome #2 Created Bool_Array: 175 layers, Predicted Fitness: 0.7880000174045563, Done
Generation #1, Crossover Genome #3 Created Bool_Array: 175 layers, Predicted Fitness: 0.7806999981403351, Done
Gen

In [205]:
def mutation(winner_bool_arr, class_instance_arr):
    
    for i, genome in enumerate(class_instance_arr):

        print('Generation #%s, Genome #%s, Predicted Fitness: %s' % (n_gen, i, genome.fitness))
        
        mutation_copy_arr = deepcopy(genome.bool_arr)

        if np.random.uniform(0,1) < PROB_MUTATION:

            flag = np.random.randint(0, len(winner_bool_arr[0]))
            flag = round(flag*PROB_MUTATION)

            mutation_arr = random.sample(range(0, len(winner_bool_arr[0])), flag)
            mutation_arr_sort = sorted(mutation_arr)
            
            for idx in mutation_arr_sort:
                boolen = mutation_copy_arr[idx]
                if boolen == True:
                    mutation_copy_arr[idx] = False
                else:
                    mutation_copy_arr[idx] = True

            genome.update_trainable(bool_arr=mutation_copy_arr)
            
            print('Generation #%s, Genome #%s, Mutation Happened!!! \t size: %s, Mutation_Array: %s, Done' % (n_gen, i, flag, mutation_arr_sort))


In [214]:
crossover_bool_arr = []
for i, genome in enumerate(crossovers):
    crossover_bool_arr.append(genome.bool_arr)
    print(crossover_bool_arr[i] is crossovers[i].bool_arr)

mutation(winner_bool_arr, crossovers)

True
True
True
True
True
Generation #1, Genome #0, Predicted Fitness: 0.784500002861023
Generation #1, Genome #1, Predicted Fitness: 0.7933000028133392
Generation #1, Genome #1, Mutation Happened!!! 	 size: 6, Mutation_Array: [10, 38, 44, 107, 119, 125], Done
Generation #1, Genome #2, Predicted Fitness: 0.7880000174045563
Generation #1, Genome #3, Predicted Fitness: 0.7806999981403351
Generation #1, Genome #4, Predicted Fitness: 0.7757000029087067


In [215]:
print(crossover_bool_arr[0] is crossovers[0].bool_arr)
print(crossover_bool_arr[1] is crossovers[1].bool_arr)
print(crossover_bool_arr[2] is crossovers[2].bool_arr)
print(crossover_bool_arr[3] is crossovers[3].bool_arr)
print(crossover_bool_arr[4] is crossovers[4].bool_arr)

True
False
True
True
True


In [227]:
# Parameters
N_POPULATION = 10
N_BEST = 6
N_CHILDREN = 5
PROB_MUTATION = 0.04

epoch = 3
batch_size =256
save_path = 'D:\GH\Audio\GA\pickle_data'

# generate 1st population
genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(N_POPULATION)]
nw_genomes = [Random_Finetune_ResNet50((32,32)) for _ in range(N_POPULATION)]

n_gen = 0

first_accuracy = np.array([])
first_bool_arr = []
for i, genome in enumerate(genomes):
    print("===== Generaton #%s\tGenome #%s : Fitness %s =====" % (n_gen, i, genome.fitness))
    # print(type(genome.bool_arr))
    # print(genome.bool_arr)
    first_accuracy = np.append(first_accuracy, genome.fitness)
    # bool_arr = np.append(bool_arr, genome.bool_arr)
    first_bool_arr.append(genome.bool_arr)

first_bool_arr = np.asarray(first_bool_arr)
data = zip(first_accuracy, first_bool_arr)

# save
filename1 = '0_Generation_Bool_Arr.pickle'
filepath = os.path.join(save_path, filename1)

with open(filepath, 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)


while True:
    n_gen += 1

    for i, genome in enumerate(genomes):

        genome = genomes[i]
        model = genome.forward(0.0001)
        history = genome.train_model(model, x_train, y_train, (x_test, y_test), epoch, batch_size)
        fitness = history.history['val_accuracy']
        sorted_fitness = sorted(fitness, reverse=True)
        genome.fitness = sorted_fitness[0]

        print('Generation #%s, Genome #%s, Fitness: %s, Best Fitness: %s' % (n_gen, i, fitness, genome.fitness))

    for i, genome in enumerate(genomes):
        print("===== Generaton #%s\t%s th Fitness %s =====" % (n_gen, i, genomes[i].fitness))

    # if best_genomes is not None:
    #     genomes.extend(best_genomes)
    genomes.sort(key=lambda x: x.fitness, reverse=True)

    print('===== Generaton #%s\tBest Fitness %s =====' % (n_gen, genomes[0].fitness))

    # 우성 genomes 를 만드는 과정
    accuracy = np.array([])
    bool_arr = []

    for i, genome in enumerate(genomes):
        print("===== Generaton #%s\tGenome #%s : Fitness %s =====" % (n_gen, i, genome.fitness))
        # print(type(genome.bool_arr))
        # print(genome.bool_arr)
        accuracy = np.append(accuracy, genome.fitness)
        # bool_arr = np.append(bool_arr, genome.bool_arr)
        bool_arr.append(genome.bool_arr)

    bool_arr = np.asarray(bool_arr)

    print(" 우성 genome에 대한 Freezing, Trainable Layer 서치 완료 ")
    # print(accuracy.shape)
    # print(bool_arr.shape)

    # 우성 bool_arr
    winner_acc = accuracy[:N_BEST]
    winner_bool_arr = bool_arr[:N_BEST]

    # CROSSOVER with Sequantial
    crossover_sequentail(N_CHILDREN, winner_acc, winner_bool_arr, nw_genomes[:N_CHILDREN])
    crossover_sequentail(N_CHILDREN, winner_acc, winner_bool_arr, nw_genomes[N_CHILDREN:])

    # mutation
    mutation(winner_bool_arr, nw_genomes)

    # 유전 연산 결과를 업데이트하는 과정
    process_accuracy = np.array([])
    process_bool_arr = []
    for i, genome in enumerate(nw_genomes):
        print("===== Generaton #%s\tGenome #%s : Fitness %s =====" % (n_gen, i, genome.fitness))
        # print(type(genome.bool_arr))
        # print(genome.bool_arr)
        process_accuracy = np.append(process_accuracy, genome.fitness)
        # bool_arr = np.append(bool_arr, genome.bool_arr)
        process_bool_arr.append(genome.bool_arr)

    process_bool_arr = np.asarray(process_bool_arr)

    print(" 유전연산에 대한 Freezing, Trainable Layer 서치 완료 및 Next Generation 준비 ")
    # print(process_accuracy.shape)
    # print(process_bool_arr.shape)

    for i, genome in enumerate(genomes):
        genome.update_trainable(bool_arr=process_bool_arr[i])
        genome.fitness = process_accuracy[i]

    # genomes.sort(key=lambda x: x.fitness, reverse=True)
    print('Generation #%s, Done' % (n_gen))
    data = zip(process_accuracy, process_bool_arr)

    # save
    filename = str(n_gen) + '_Generation_Bool_Arr.pickle'
    filepath = os.path.join(save_path, filename)

    with open(filepath, 'wb') as f:
        pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

===== Generaton #0	Genome #0 : Fitness 0 =====
===== Generaton #0	Genome #1 : Fitness 0 =====
===== Generaton #0	Genome #2 : Fitness 0 =====
===== Generaton #0	Genome #3 : Fitness 0 =====
===== Generaton #0	Genome #4 : Fitness 0 =====
===== Generaton #0	Genome #5 : Fitness 0 =====
===== Generaton #0	Genome #6 : Fitness 0 =====
===== Generaton #0	Genome #7 : Fitness 0 =====
===== Generaton #0	Genome #8 : Fitness 0 =====
===== Generaton #0	Genome #9 : Fitness 0 =====
Epoch 1/3
235/235 [==============================] - 23s 74ms/step - loss: 1.9333 - accuracy: 0.2751 - val_loss: 1.0246 - val_accuracy: 0.6058
Epoch 2/3
235/235 [==============================] - 16s 68ms/step - loss: 0.9759 - accuracy: 0.6156 - val_loss: 0.9328 - val_accuracy: 0.6353
Epoch 3/3
235/235 [==============================] - 16s 68ms/step - loss: 0.7862 - accuracy: 0.6940 - val_loss: 0.8540 - val_accuracy: 0.6709
Generation #1, Genome #0, Fitness: [0.6057999730110168, 0.6352999806404114, 0.6708999872207642], Best

In [ ]:
# 모델 파라미터를 저장하지 않으면 deepcopy가 진행되지 않는 것 같음

# best_genomes = deepcopy(genomes[:1])

# WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
# INFO:tensorflow:Assets written to: ram://d7898afe-6f2d-4baa-8dcd-cccd42f17e16/assets
# FileNotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for ram://7fd86df0-a48d-49a8-955e-b376083fbc1e/variables/variables
# You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` 
# to the io_device such as '/job:localhost'.

In [ ]:
import pickle

save_path = 'D:\GH\Audio\GA\pickle_data'
filename1 = '1st_Generation_Bool_Arr.pickle'
filename2 = '1st_Generation_Bool_Arr.pickle'

filepath = os.path.join(save_path, filename1)


# save
with open(filepath, 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

# save
with open(filepath, 'wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)